In [27]:
import requests
from datetime import timedelta, date, datetime
import json
import pandas as pd
import os
import csv

In [28]:
def daterange(start_date):
    end_date = datetime.now() - timedelta(days=1)
    current_date = datetime(start_date.year, start_date.month, start_date.day)
    one_day = timedelta(days=1)
    while current_date <= end_date:
        if current_date.month not in [1, 2, 11, 12]:
            yield current_date
        current_date += one_day

In [36]:
def get_daily_game_ids(api_key, year, month, day):
    ''' Fetches game IDs from the daily summary endpoint. '''
    try:
        # Ensure month and day are integers and format them to two digits
        month_str = f"{int(month):02d}"
        day_str = f"{int(day):02d}"
    except ValueError:
        raise ValueError("Month and Day must be convertible to integers.")

    # Construct the URL based on provided parameters
    url = f"https://api.sportradar.com/mlb/trial/v7/en/games/{year}/{month_str}/{day_str}/summary.json"
    
    # Set headers to define expected response format
    headers = {'Accept': 'application/json'}
    
    # Send the API request
    response = requests.get(url, params={'api_key': api_key}, headers=headers)
    
    # Debug print to check URL and response status
    print(f"Request URL: {url} | Status Code: {response.status_code}")

    if response.status_code == 200:
        data = response.json()
        games = data.get('league', {}).get('games', [])
        print(f"Games found on {year}-{month_str}-{day_str}: {len(games)}")
        return [game['game']['id'] for game in games]
    elif response.status_code >= 500:
        print("Server error, consider retrying the request")
        return []
    else:
        print(f"Failed to retrieve data: Status Code {response.status_code}")
        return []

In [41]:
def extract_team_pitching_stats(team_data, team_type, stats_list, custom_game_id):
    # Access team-level pitching statistics
    team_stats = team_data.get(team_type, {}).get('statistics', {}).get('pitching', {})

    if team_stats:
        ip = team_stats['overall'].get('ip_2', '0')  # Innings pitched
        
        stats_list.append({
            'home_away': team_type,
            'ip': ip,
            'game_id': custom_game_id
        })
        
    return stats_list

In [44]:
def fetch_and_store_pitching_stats(api_key, start_date):
    team_stats = []  # Initialize the list outside the for loop to maintain scope throughout the function
    
    for current_date in daterange(start_date):
        year = current_date.year
        month = current_date.month
        day = current_date.day
        game_ids = get_daily_game_ids(api_key, year, month, day)
        for game_id in game_ids[:1]:  # Limiting to the first game for testing or example purposes
            url = f'https://api.sportradar.com/mlb/trial/v7/en/games/{game_id}/extended_summary.json?api_key={api_key}'
            response = requests.get(url)

            if response.status_code == 200:
                data = response.json()
                game_info = data['game']
                date = datetime.strptime(game_info['scheduled'], "%Y-%m-%dT%H:%M:%S%z").strftime("%Y%m%d")
                day_night = game_info['day_night']
                custom_game_id = f"{game_info['home']['abbr']}{game_info['away']['abbr']}{date}{day_night}"

                # Extract team stats for home and away
                extract_team_pitching_stats(data['game'], 'home', team_stats, custom_game_id)
                extract_team_pitching_stats(data['game'], 'away', team_stats, custom_game_id)
            else:
                print(f"Failed to fetch data: {response.status_code} {response.text}")

        break  # Optionally, break after the first date for testing


    # Define the path for the file
    file_path = '../../data/pitching-test.csv'

    # Ensure the directory exists, create if not
    os.makedirs(os.path.dirname(file_path), exist_ok=True)

    # Writing to CSV
    with open(file_path, 'w', newline='') as csvfile:
        fieldnames = [
            'home_away',
            'ip',
            'game_id'
        ]

        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for stat in team_stats:
            writer.writerow(stat)

        print(f"Team pitching data written to '{file_path}'")


In [45]:
api_key = '95w69RTFvT2czN4vnvoLnW8Os95JWg4SBFjJF1f0'  # Replace with your actual API key
start_date = datetime(2024, 5, 5).date()
fetch_and_store_pitching_stats(api_key, start_date)

Request URL: https://api.sportradar.com/mlb/trial/v7/en/games/2024/05/05/summary.json | Status Code: 200
Games found on 2024-05-05: 15
Team pitching data written to '../../data/pitching-test.csv'
